# Protocol Data Blob 

Goal: load protocol data blob in from matlab & unpack

Protocol Data structure for DMS:

| variable             | values          | key                                                 |
|----------------------|-----------------|-----------------------------------------------------|
| `hit_history`        | [0, 1, NaN]     | [error, hit, violation]                             |
| `temp_error_history` | [0, 1, NaN]     | [no terror, terror, violation]                      |
| `side_history`       | [l, r]          | [left, right]                                       |
| `result_history`     | [1, 2, 3, 4, 5] | [hit, error, violation, terror, crash]              |
| `sa`                 | float           | value of sa in Hz                                   |
| `sb`                 | float           | value of sb in Hz                                   |
| `dms_type`           | [0, 1]          | [nonmatch, match]                                   |
| `delay`              | float           | duration of delay between sa/sb in s                |
| `fixation`           | float           | duration of pre --> go (fixation required or not)   |
| `timeouts`           | float           | duration of timeout in s                            |
| `helper`             | [0, 1]          | if trial was in helper block (not in use currently) |
| `stage`              | float           | stage number                                        |


In [188]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import os
import seaborn as sns
import pandas as pd
from pathlib import Path

from io_utils import *

# settings
%reload_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read
 

### Single Session

In [189]:
# grab protocol data from a session
bdata = dj.create_virtual_module('new_acquisition', 'bdatatest')
session_key = {'sessid': 880487}
session_data = (bdata.Sessions & session_key).fetch('protocol_data', as_dict=True)

In [194]:
animal_id = 'R500'
sessid =  session_key['sessid']
date = "2022-07-19"
protocol_dict = convert_to_dict(session_data)
protocol_df = make_protocol_df(protocol_dict, animal_id, [date], [sessid])

protocol_df.head()

,trial,animal_id,date,sessid,hits,temperror,sides,result,sa,sb,dms_type,delay,fixation,timeouts,helper,stage
0,1,R500,880487,2022-07-19,1,0,r,1,12.0,3.0,False,0.1,1.100000,NaN,0,4
1,2,R500,880487,2022-07-19,<NA>,<NA>,r,3,12.0,3.0,False,0.1,0.455246,0.500,0,4
2,3,R500,880487,2022-07-19,<NA>,<NA>,r,3,12.0,3.0,False,0.1,0.455246,0.525,0,4
3,4,R500,880487,2022-07-19,0,0,l,2,12.0,12.0,True,0.1,0.455246,0.500,0,4
4,5,R500,880487,2022-07-19,<NA>,<NA>,r,3,3.0,12.0,False,0.1,0.460230,0.550,0,4


### Multi Session

In [196]:
dfs = fetch_latest_protocol_data(['R500'], as_dict=False)
dfs[0].head()

fetched 24 sessions for R500


,trial,animal_id,date,sessid,hits,temperror,sides,result,sa,sb,dms_type,delay,fixation,timeouts,helper,stage
0,1,R500,2022-07-19,879474,0,1,l,4,12.0,12.0,True,0.1,1.1,0.1,0,1
1,2,R500,2022-07-19,879474,0,1,l,4,12.0,12.0,True,0.1,0.5,0.1,0,1
2,3,R500,2022-07-19,879474,1,0,r,1,12.0,3.0,False,0.1,0.5,NaN,0,1
3,4,R500,2022-07-19,879474,1,0,r,1,3.0,12.0,False,0.1,0.5,NaN,0,1
4,5,R500,2022-07-19,879474,0,1,r,4,12.0,3.0,False,0.1,0.5,0.1,0,1


In [198]:
dfs = fetch_latest_protocol_data(['R500', 'R600'], as_dict=True)
print(dfs.keys())
dfs['R500'].head()

fetched 24 sessions for R500
fetched 25 sessions for R600
dict_keys(['R500', 'R600'])


,trial,animal_id,date,sessid,hits,temperror,sides,result,sa,sb,dms_type,delay,fixation,timeouts,helper,stage
0,1,R500,2022-07-19,879474,0,1,l,4,12.0,12.0,True,0.1,1.1,0.1,0,1
1,2,R500,2022-07-19,879474,0,1,l,4,12.0,12.0,True,0.1,0.5,0.1,0,1
2,3,R500,2022-07-19,879474,1,0,r,1,12.0,3.0,False,0.1,0.5,NaN,0,1
3,4,R500,2022-07-19,879474,1,0,r,1,3.0,12.0,False,0.1,0.5,NaN,0,1
4,5,R500,2022-07-19,879474,0,1,r,4,12.0,3.0,False,0.1,0.5,0.1,0,1


#### scraps

In [157]:
for key in protocol_dict.keys():
    print(f"{key} type:{type(protocol_dict[key][0])} ")

hits type:<class 'numpy.float64'> 
temperror type:<class 'numpy.float64'> 
sides type:<class 'str'> 
result type:<class 'numpy.float64'> 
sa type:<class 'numpy.float64'> 
sb type:<class 'numpy.float64'> 
dms_type type:<class 'numpy.bool_'> 
delay type:<class 'numpy.float64'> 
fixation type:<class 'numpy.float64'> 
timeouts type:<class 'numpy.float64'> 
helper type:<class 'numpy.float64'> 
stage type:<class 'numpy.float64'> 


In [156]:
for key,value in protocol_dict.items():

    `print(f"{key} length:{len(value)}")

hits length:45
temperror length:45
sides length:45
result length:45
sa length:45
sb length:45
dms_type length:45
delay length:45
fixation length:45
timeouts length:45
helper length:45
stage length:45


Additional functions:
* read in pd data for an animal or default group
* or maybe have a load_last lag for the fetch_lastest

VARS:
SAVE_DIR
ANIMAL_IDS

Order:

fetch_latest_protocol_data

convert_to_dict 
* takes blob(s)
* calls mym_blob_to_dict
* returns list of dictonaries

make_protocol_df
* takes dicts, animal id, sessions, dates
* calls correct_sb
* calls clean_protocol_df
* returns list of dataframes

saves out animals data frame as csv

by defualt returns a dictonary with animal name as keys, can also return a list of df instead




In [181]:
as_dict = True
x = 7
[x + 3]  if as_dict is True else x - 3

x

7